# Mapping the Latent Space of CXR Patient Geometry
## Zebrastack V0 VAE trained on the CovidNet chest radiograph dataset
The zebrastack is a variational auto-encoder that is _very_ roughly aligned with the distributed hierarchical architecture + shifter circuit.  Here it is applied to recognition of the CovidNet chest radiograph dataset.

First, declare some helper pyplot functions to show us thumbnails.

In [1]:
# %load show_original_decoded.py
import matplotlib.pyplot as plt
def show_grayscale(rows, columns, at, pixel_array, sz):
    import scipy
    ax = plt.subplot(rows, columns, at)
    interp_array = scipy.ndimage.zoom(pixel_array.reshape(sz,sz), 4.0, order=5)
    plt.imshow(interp_array.reshape(sz*4, sz*4), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

def show_original_decoded(original, decoded, sz, n=10):
    plt.figure(figsize=(n*2, 4))
    for i in range(n):
        show_grayscale(2, n, i+1, original[i], sz)
        show_grayscale(2, n, i+1+n, decoded[i], sz)
    plt.show(block=True)

Now load the center-surround processed images

In [2]:
# now reload
from pathlib import Path
import os
import time

start_time = time.time()

clahe_temp_path = Path(os.environ['DATA_TEMP']) / 'anat0mixer_temp' / 'clahe_processed'
processed_imgs = {}
for npy_filepath in clahe_temp_path.glob('*.npy'):
    import numpy as np
    processed_imgs[npy_filepath.stem] = np.load(npy_filepath)
end_time = time.time()

print(f"Loaded {len(processed_imgs)} npy in {end_time - start_time} seconds")

Loaded 60361 npy in 120.67739701271057 seconds


Determine shape of processed images

In [3]:
one_img = list(processed_imgs.values())[0]
sz = one_img.shape[0]
processed_imgs = {name:np.reshape(processed_imgs[name], (sz,sz,1)) for name in processed_imgs}
print(f"Processed images are {sz}x{sz}x1")

Processed images are 128x128x1


Partition the data in to train and test: 90% train and 10% test.  We are assuming all images are from distinct patients, so there is no need to partition by subject.

In [ ]:
test_count = int(len(processed_imgs)/10)
processed_img_list = list(processed_imgs.values())
x_test = processed_img_list[:test_count]
x_train = processed_img_list[test_count:]
x_train, x_test = np.array(x_train), np.array(x_test)
print(f"Training data has shape {x_train.shape}")
print(f"Testing data has shape {x_test.shape}")

## Defining the Variational Autoencoder
To defined the variational autoencoder, we need to helper functions:
* A function for reparameterized sampling
* A function for KLDiv loss

In [4]:
# %load zebrastack_v0_model.py
from keras.layers import Dense, Input, SpatialDropout2D
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import LocallyConnected2D, ZeroPadding2D
from keras.layers import MaxPooling2D, UpSampling2D
from keras.layers import Reshape, Conv2DTranspose
from keras.layers import ActivityRegularization
from keras.models import Model
from keras.utils import plot_model
from keras import backend as K

sz = 128
latent_dim = 12
locally_connected_channels = 2
do_plot_model = False
cit_decimate = False
act_func = 'softplus' # or 'relu'
use_mse = True # False

def sampling(args):
    """
    Reparameterization trick by sampling fr an isotropic unit Gaussian.
    instead of sampling from Q(z|X), sample eps = N(0,I) 
        then z = z_mean + sqrt(var)*eps    
    # Arguments
        args (tensor tuple): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """    
    z_mean, z_log_var = args
    
    from keras import backend as K
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def vae_loss(z_mean, z_log_var, y_true, y_pred):
    """
    Compute VAE loss, using either mse or crossentropy.
    # Arguments
        z_mean: mean of Q(z|X)
        z_log_var: log variance of Q(z|X)
        y_true, y_pred: truth and predicated values
    # Returns
        loss value
    """
    from tensorflow.keras.losses import mse, binary_crossentropy
    from keras import backend as K
    img_pixels = sz * sz
    if use_mse:
        match_loss = mse(K.flatten(y_true), K.flatten(y_pred)) * img_pixels
    else:
        match_loss = binary_crossentropy(K.flatten(y_true), K.flatten(y_pred)) * img_pixels
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(match_loss + kl_loss)

def create_encoder():
    """
    """
    # create encoder side
    retina = Input(shape=(sz,sz,1), name='retina_{}'.format(sz))

    v1_conv2d = Conv2D(16, (5,5), name='v1_conv2d', activation=act_func, padding='same')(retina)
    v1_maxpool = MaxPooling2D((2,2), name='v1_maxpool', padding='same')(v1_conv2d)
    v1_dropout = SpatialDropout2D(0.1, name='v1_dropout')(v1_maxpool)

    v2_conv2d = Conv2D(16, (3,3), name='v2_conv2d', activation=act_func, padding='same')(v1_dropout)
    v2_maxpool = MaxPooling2D((2,2), name='v2_maxpool', padding='same')(v2_conv2d)

    v4_conv2d = Conv2D(32, (3,3), name='v4_conv2d', activation=act_func, padding='same')(v2_maxpool)
    v4_maxpool = MaxPooling2D((2,2), name='v4_maxpool', padding='same')(v4_conv2d)

    pit_conv2d = Conv2D(32, (3,3), name='pit_conv2d', activation=act_func, padding='same')(v4_maxpool)
    pit_maxpool = MaxPooling2D((2,2), name='pit_maxpool', padding='same')(pit_conv2d)

    cit_conv2d = Conv2D(64, (3,3), name='cit_conv2d', activation=act_func, padding='same')(pit_maxpool)
    if cit_decimate:
        cit_maxpool = MaxPooling2D((2,2), name='cit_maxpool', padding='same')(cit_conv2d)
        cit_out = cit_maxpool
    else:
        cit_out = cit_conv2d

    ait_local = LocallyConnected2D(locally_connected_channels, (3,3), 
                                   name='ait_local', activation=act_func)(cit_out)
    # ait_padding = ZeroPadding2D(padding=(1,1), name='ait_padding')(ait_local)
    # x = MaxPooling2D((2,2), padding='same', name='ait_maxpool')(ait_padding)

    ait_regular = ActivityRegularization(l1=0.0e-4, l2=0.0e-4, name='ait_regular')(ait_local)

    # shape info needed to build decoder model
    shape = K.int_shape(ait_regular)
    # print(shape)

    # generate latent vector Q(z|X)
    pulvinar_flatten = Flatten(name='pulvinar_flatten')(ait_regular)
    pulvinar_dense = Dense(latent_dim, activation=act_func, name='pulvinar_dense')(pulvinar_flatten)
    z_mean = Dense(latent_dim, name='z_mean')(pulvinar_dense)
    z_log_var = Dense(latent_dim, name='z_log_var')(pulvinar_dense)

    # use reparameterization trick to push the sampling out as input
    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    encoder = Model(retina, [z_mean, z_log_var, z], name='v1_to_pulvinar_encoder')
    encoder.summary()
    
    if do_plot_model: 
        plot_model(encoder, to_file='data\{}.png'.format(encoder.name), show_shapes=True)

    return retina, encoder, shape, [z_mean, z_log_var, z]

def create_decoder(shape):
    """
    """
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    pulvinar_dense_back = Dense(shape[1] * shape[2] * shape[3], name='pulvinar_dense_back', 
                                activation=act_func)(latent_inputs)
    pulvinar_antiflatten = Reshape((shape[1], shape[2], shape[3]), name='pulvinar_antiflatten')(pulvinar_dense_back)

    ait_padding_back = ZeroPadding2D(padding=(1,1), name='ait_padding_back')(pulvinar_antiflatten)
    ait_local_back = LocallyConnected2D(locally_connected_channels, (3,3), name='ait_local_back', 
                                        activation=act_func)(ait_padding_back)

    cit_padding_back = ZeroPadding2D(padding=(1,1), name='cit_padding_back')(ait_local_back)
    if cit_decimate:
        ait_upsample_back = UpSampling2D((2,2), name='ait_upsample_back')(cit_padding_back)
        ait_out_back = ait_upsample_back
    else:
        ait_out_back = cit_padding_back

    cit_conv2d_trans = Conv2DTranspose(64, (3,3), name='cit_conv2d_trans', 
                                       activation=act_func, padding='same')(ait_out_back)
    cit_upsample_back = UpSampling2D((2,2), name='cit_upsample_back')(cit_conv2d_trans)

    pit_conv2d_trans = Conv2DTranspose(32, (3,3), name='pit_conv2d_trans', 
                                       activation=act_func, padding='same')(cit_upsample_back)
    pit_upsample_back = UpSampling2D((2,2), name='pit_upsample_back')(pit_conv2d_trans)

    v4_conv2d_trans = Conv2DTranspose(32, (3,3), name='v4_conv2d_trans', 
                                      activation=act_func, padding='same')(pit_upsample_back)
    v4_upsample_back = UpSampling2D((2,2), name='v4_upsample_back')(v4_conv2d_trans)

    v2_conv2d_trans = Conv2DTranspose(16, (3,3), name='v2_conv2d_trans', 
                                      activation=act_func, padding='same')(v4_upsample_back)
    v2_upsample_back = UpSampling2D((2,2), name='v2_upsample_back')(v2_conv2d_trans)

    v1_conv2d_5x5_back = Conv2D(1, (5,5), name='v1_conv2d_5x5_back', 
                                activation='sigmoid', padding='same')(v2_upsample_back)
    decoder = Model(latent_inputs, v1_conv2d_5x5_back, name='pulvinar_to_v1_decoder')
    decoder.summary()
    if do_plot_model: 
        plot_model(decoder, to_file='data\{}.png'.format(decoder.name), show_shapes=True)
        
    return decoder

def create_autoencoder(retina, encoder, prob_model, decoder):
    """
    """
    autoencoder_output = decoder(encoder(retina)[2])
    autoencoder = Model(retina, autoencoder_output, name='v1_to_pulvinar_vae')

    # now compile with the optimizer VAE loss function
    optimizer = 'adadelta'
    [z_mean, z_log_var, z] = prob_model
    autoencoder.compile(optimizer=optimizer, 
                        loss=lambda y_true, y_pred: vae_loss(z_mean, z_log_var, y_true, y_pred))
    autoencoder.summary()
    if do_plot_model: 
        plot_model(autoencoder, to_file='data\{}.png'.format(autoencoder.name), show_shapes=True)
        
    return autoencoder

Using TensorFlow backend.


In [5]:
retina, encoder, latent_shape, [z_mean, z_log_var, z] = create_encoder()
decoder = create_decoder(latent_shape)
autoencoder = create_autoencoder(retina, encoder, [z_mean, z_log_var, z], decoder)

Model: "v1_to_pulvinar_encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
retina_128 (InputLayer)         (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
v1_conv2d (Conv2D)              (None, 128, 128, 16) 416         retina_128[0][0]                 
__________________________________________________________________________________________________
v1_maxpool (MaxPooling2D)       (None, 64, 64, 16)   0           v1_conv2d[0][0]                  
__________________________________________________________________________________________________
v1_dropout (SpatialDropout2D)   (None, 64, 64, 16)   0           v1_maxpool[0][0]                 
_____________________________________________________________________________

In [ ]:
def sampling(args):
    """
    Reparameterization trick by sampling fr an isotropic unit Gaussian.
    instead of sampling from Q(z|X), sample eps = N(0,I) 
        then z = z_mean + sqrt(var)*eps    
    # Arguments
        args (tensor tuple): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """    
    z_mean, z_log_var = args
    
    from keras import backend as K
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
use_mse = True # False
def vae_loss(z_mean, z_log_var, y_true, y_pred):
    """
    Compute VAE loss, using either mse or crossentropy.
    # Arguments
        z_mean: mean of Q(z|X)
        z_log_var: log variance of Q(z|X)
        y_true, y_pred: truth and predicated values
    # Returns
        loss value
    """
    from tensorflow.keras.losses import mse, binary_crossentropy
    from keras import backend as K
    img_pixels = sz * sz
    if use_mse:
        match_loss = mse(K.flatten(y_true), K.flatten(y_pred)) * img_pixels
    else:
        match_loss = binary_crossentropy(K.flatten(y_true), K.flatten(y_pred)) * img_pixels
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(match_loss + kl_loss)

We create the Keras model in three parts: 
* encode_only: the recognition subsystem(encoder)
* decode_only: decoder
* autoencoder: the full system

Parameters that control the model are:
* sz: size x size of inputs
* latent_dim: latent dimension at the pulvinar

The layers are named to suggest correspondence between each layer and an area in the distributed hierarchy model / shifter circuit.

In [ ]:
from keras.layers import Dense, Input, SpatialDropout2D
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import LocallyConnected2D, ZeroPadding2D
from keras.layers import MaxPooling2D, UpSampling2D
from keras.layers import Reshape, Conv2DTranspose
from keras.layers import ActivityRegularization
from keras import backend as K

sz = 128
latent_dim = 12
do_plot_model = False
cit_decimate = False
act_func = 'softplus' # or 'relu'

# create encoder side
retina = Input(shape=(sz,sz,1), name='retina_{}'.format(sz))

v1_conv2d = Conv2D(16, (5,5), name='v1_conv2d', activation=act_func, padding='same')(retina)
v1_maxpool = MaxPooling2D((2,2), name='v1_maxpool', padding='same')(v1_conv2d)
v1_dropout = SpatialDropout2D(0.1, name='v1_dropout')(v1_maxpool)

v2_conv2d = Conv2D(16, (3,3), name='v2_conv2d', activation=act_func, padding='same')(v1_dropout)
v2_maxpool = MaxPooling2D((2,2), name='v2_maxpool', padding='same')(v2_conv2d)

v4_conv2d = Conv2D(32, (3,3), name='v4_conv2d', activation=act_func, padding='same')(v2_maxpool)
v4_maxpool = MaxPooling2D((2,2), name='v4_maxpool', padding='same')(v4_conv2d)

pit_conv2d = Conv2D(32, (3,3), name='pit_conv2d', activation=act_func, padding='same')(v4_maxpool)
pit_maxpool = MaxPooling2D((2,2), name='pit_maxpool', padding='same')(pit_conv2d)

cit_conv2d = Conv2D(64, (3,3), name='cit_conv2d', activation=act_func, padding='same')(pit_maxpool)
if cit_decimate:
    cit_maxpool = MaxPooling2D((2,2), name='cit_maxpool', padding='same')(cit_conv2d)
    cit_out = cit_maxpool
else:
    cit_out = cit_conv2d
    
locally_connected_channels = 2    
ait_local = LocallyConnected2D(locally_connected_channels, (3,3), 
                               name='ait_local', activation=act_func)(cit_out)
# ait_padding = ZeroPadding2D(padding=(1,1), name='ait_padding')(ait_local)
# x = MaxPooling2D((2,2), padding='same', name='ait_maxpool')(ait_padding)

ait_regular = ActivityRegularization(l1=0.0e-4, l2=0.0e-4, name='ait_regular')(ait_local)

# shape info needed to build decoder model
shape = K.int_shape(ait_regular)
# print(shape)

# generate latent vector Q(z|X)
pulvinar_flatten = Flatten(name='pulvinar_flatten')(ait_regular)
pulvinar_dense = Dense(latent_dim, activation=act_func, name='pulvinar_dense')(pulvinar_flatten)
z_mean = Dense(latent_dim, name='z_mean')(pulvinar_dense)
z_log_var = Dense(latent_dim, name='z_log_var')(pulvinar_dense)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

from keras.models import Model
encoder = Model(retina, [z_mean, z_log_var, z], name='v1_to_pulvinar_encoder')
encoder.summary()
    
from keras.utils import plot_model
if do_plot_model: 
    plot_model(encoder, to_file='data\{}.png'.format(encoder.name), show_shapes=True)

The decoder is defined as the reverse layers of the encoder.

In [ ]:
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
pulvinar_dense_back = Dense(shape[1] * shape[2] * shape[3], name='pulvinar_dense_back', 
                            activation=act_func)(latent_inputs)
pulvinar_antiflatten = Reshape((shape[1], shape[2], shape[3]), name='pulvinar_antiflatten')(pulvinar_dense_back)

ait_padding_back = ZeroPadding2D(padding=(1,1), name='ait_padding_back')(pulvinar_antiflatten)
ait_local_back = LocallyConnected2D(locally_connected_channels, (3,3), name='ait_local_back', 
                                    activation=act_func)(ait_padding_back)

cit_padding_back = ZeroPadding2D(padding=(1,1), name='cit_padding_back')(ait_local_back)
if cit_decimate:
    ait_upsample_back = UpSampling2D((2,2), name='ait_upsample_back')(cit_padding_back)
    ait_out_back = ait_upsample_back
else:
    ait_out_back = cit_padding_back
    
cit_conv2d_trans = Conv2DTranspose(64, (3,3), name='cit_conv2d_trans', 
                                   activation=act_func, padding='same')(ait_out_back)
cit_upsample_back = UpSampling2D((2,2), name='cit_upsample_back')(cit_conv2d_trans)

pit_conv2d_trans = Conv2DTranspose(32, (3,3), name='pit_conv2d_trans', 
                                   activation=act_func, padding='same')(cit_upsample_back)
pit_upsample_back = UpSampling2D((2,2), name='pit_upsample_back')(pit_conv2d_trans)

v4_conv2d_trans = Conv2DTranspose(32, (3,3), name='v4_conv2d_trans', 
                                  activation=act_func, padding='same')(pit_upsample_back)
v4_upsample_back = UpSampling2D((2,2), name='v4_upsample_back')(v4_conv2d_trans)

v2_conv2d_trans = Conv2DTranspose(16, (3,3), name='v2_conv2d_trans', 
                                  activation=act_func, padding='same')(v4_upsample_back)
v2_upsample_back = UpSampling2D((2,2), name='v2_upsample_back')(v2_conv2d_trans)

v1_conv2d_5x5_back = Conv2D(1, (5,5), name='v1_conv2d_5x5_back', 
                            activation='sigmoid', padding='same')(v2_upsample_back)
decoder = Model(latent_inputs, v1_conv2d_5x5_back, name='pulvinar_to_v1_decoder')
decoder.summary()
if do_plot_model: 
    plot_model(decoder, to_file='data\{}.png'.format(decoder.name), show_shapes=True)

Finally put it together to the full model, and compile with the desired optimizer

In [ ]:
autoencoder_output = decoder(encoder(retina)[2])
autoencoder = Model(retina, autoencoder_output, name='v1_to_pulvinar_vae')

# now compile with the optimizer VAE loss function
optimizer = 'adadelta'
autoencoder.compile(optimizer=optimizer, 
                    loss=lambda y_true, y_pred: vae_loss(z_mean, z_log_var, y_true, y_pred))
autoencoder.summary()
if do_plot_model: 
    plot_model(autoencoder, to_file='data\{}.png'.format(autoencoder.name), show_shapes=True)

## Pre-Training the model
Train by calling .fit with the training data.  
* Batch size of 512 helps convergence, but causes resource exhaustion > 128x128.
* Epochs > 1024 take a while, but tend to lead to better results

In [ ]:
autoencoder.fit(x_train[:5000], x_train[:5000], 
                epochs=128, batch_size=128, 
                shuffle=True, validation_data=(x_test[:500], x_test[:500]))

## Inspecting Reconstructing Images
To qualitatively assess the model, look at a few images and their reconstructions from latent vectors

In [ ]:
from functools import lru_cache
import numpy as np

# @lru_cache(maxsize=None)
def decode_latent(decoder, latent_vector:tuple):
    """
    helper to decode latent, with caching to speed up
    # Arguments
        decode_only: model for decoding
        latent_vector: _tuple_ representing the vector to be decoded
    # Returns
        decoded image
    """
    latent_vector_arr = np.array([latent_vector])
    return decoder.predict(latent_vector_arr)

In [ ]:
from tensorflow import keras
reload_models = False
if reload_models:
    encoder = keras.models.load_model('data\zebrastack_v0_covidnet_encoder_model')
    decoder = keras.models.load_model('data\zebrastack_v0_covidnet_decoder_model')

In [ ]:
encode_only_imgs = encoder.predict(x_test[:10])
show_histo = False
if show_histo:
    for n in range(10):
        print("shape of encoded = ", encode_only_imgs[2].shape)
        hist, bins = np.histogram(encode_only_imgs[2])
        print(hist)
        print(bins)
decoded_imgs = \
    [decode_latent(decoder, tuple(latent_vector)) 
         for latent_vector in encode_only_imgs[2]]
show_original_decoded(x_test[:10], decoded_imgs, sz)

In [ ]:
import tensorflow as tf
autoencoder.save_weights('E:\\Data\\anat0mixer_temp\\202009011028_autoencoder_v0')
encoder.save_weights('E:\\Data\\anat0mixer_temp\\202009011028_encoder_v0')
decoder.save_weights('E:\\Data\\anat0mixer_temp\\202009011028_decoder_v0')

In [7]:
import tensorflow as tf
from pathlib import Path
from os import environ
anatomixer_temp = Path(environ['DATA_TEMP']) / 'anat0mixer_temp'
autoencoder.load_weights(anatomixer_temp / '202009011028_autoencoder_v0')
encoder.load_weights(anatomixer_temp / '202009011028_encoder_v0')
decoder.load_weights(anatomixer_temp / '202009011028_decoder_v0')

## Apply Shifter to Dataset

In [14]:
use_rotation, use_shear = False, False
dump = False
weight_prior = 1.0/1000.0
count_imgs = 0

def xform_params(x):
    import math
    params = {'tx':x[0], 'ty':x[1], 'zx':math.exp(x[2]/30), 'zy':math.exp(x[3]/30)}
    if use_rotation:
        params['theta'] = x[4]
    if use_shear:
        params['shear'] = x[5]
    return params

def xform_image(params, img):
    from tensorflow.keras.preprocessing.image import apply_affine_transform as transform
    return transform(img, **params, fill_mode='nearest')

def xform_prior_logsq(x):
    theta_prior = x[0] * x[0]
    shear_prior = x[5] * x[5]
    scale_prior = x[3] * x[3] + x[4] * x[4]
    return theta_prior + shear_prior + scale_prior

def xformed_latent(x, img):
    params = xform_params(x)
    xformed = xform_image(params, img)
    latent_vector = encoder.predict([[xformed]])[2]
    # prior = xform_prior_logsq(x)
    return np.linalg.norm(latent_vector) # + weight_prior * prior
    
def shift(img):
    from scipy.optimize import minimize
    import pprint
    global count_imgs
    res = minimize(xformed_latent, np.zeros((4,)),
                   method='powell', 
                   # method='nelder-mead', 
                   args=img,
                   options={'xtol': 1e-6, 
                            'ftol': 1e-6,
                            # 'maxiter': 10000,
                            'disp': dump})
    params = xform_params(res.x)
    if dump:
        print(res)
        pprint.pprint(params)
    count_imgs = 1 + count_imgs
    print('{0}: {1},{2}'.format(count_imgs, res.x[0], res.x[1]), end = '\r')
    return xform_image(params, img)

if False:
    x_train_shifted = []
    for img in x_train:
        x_train_shifted.append(shift(img))
    x_test_shifted = []
    for img in x_test:
        x_test_shifted.append(shift(img))
    show_original_decoded(x_test[:10], x_test_shifted[:10], sz)
else:
    # processed_imgs_shifted = {}
    for name in processed_imgs:
        img = processed_imgs[name]
        if name not in processed_imgs_shifted:
            processed_imgs_shifted[name] = shift(img)
            print()
            print(name, end='\r')

1: 1.218991388289745,4.4610927653062795
2: -0.8488916387025949,1.7286000587493264
3: -3.683277076518591,-1.209577373607617
4: 2.4424945974298446,0.6967187310626386
5: -0.21472941283793018,3.2298665907428457
6: 0.918520758586036,4.935111372215988
7: -11.337806383291806,4.553653545490953
8: 1.767715222069942,0.5906416654292805
9: 2.0575950261480784,1.6180339748439998
10: -0.6756248682926413,0.618033974844
11: 1.915955299214601,-1.875418001111282
12: -0.42534584951646837,0.6549862253617234
13: 6.780574348922083,1.8672575789484134
14: 6.6674251380896425,0.09704779547864306
15: 3.8487705699259416,1.5977939923148177
16: 4.791592771206571,-2.0863147261148685
17: 0.9109356792651845,0.4583876273931249
18: 0.3819660155472633,0.6311896069082051
19: -0.5563495337509331,1.1661618842887482
20: -1.4037741813866167,1.1327424089700007
21: 2.743098893756706,2.2933117650455284
22: 11.50774028862991,1.1051529500873283
23: 2.497722846330385,2.947710614042691
24: -5.769914242667113,-1.165588475536039
25: 2.

195: 6.480046125606998,2.5613948258167465
196: -3.3200017586448176,3.295649795949851
197: -0.3189716228540397,0.08335545979628867
198: 1.3499340449103592,4.766898583360387
199: 2.3657247414868703,1.4671109767518984
200: -0.5529262300968534,2.374916836941273
201: -2.463531952382672,1.2274839209457842
202: 1.7155319186416866,-0.243931460164593
203: 0.6191189979630487,2.2227559467773026
204: 0.33858015349517223,-6.1703822334214555
205: 2.6799156237502957,4.1435178879562224
206: 1.0005208992433916,-2.247931817481579
207: 3.439506049976748,1.7811803464987523
208: 0.40292787855258794,18.0229928201446
209: -0.15945196082465,2.3796843479318683
210: 0.4097056580075391,0.313262922675715
211: 0.0,6.813993776964487
212: 0.09258316673498687,3.5343390454035806
213: 9.096765667018596,6.231691600420894
214: -5.893077329859383,20.92884024927477
215: 1.0663611945366867,-0.6691720954908176
216: 6.945935609725997,6.781278790907559
217: 3.1556485738468454,16.851617701052266
218: 1.085605066275224,1.9971718

387: -1.1361820712636121,-0.16753117372919435
388: 3.957060213202961,-0.37886039006429173
389: 3.5359508625898366,1.0
390: 5.993676228682142,0.9125057048204532
391: 1.0557280842103176,0.008131487235976061
392: 0.39778975589920645,-0.09012688590668953
393: 2.4030032076668446,2.0421948751767767
394: 1.5884967876915712,-2.6180339844527367
395: 0.4898904557694257,1.6223258353852332
396: -0.9264458759527425,2.9204217179668346
397: 6.449751274868237,2.573374337241244
398: 0.27733320959905927,-0.6281251780440515
399: 3.9264915222337997,3.306575852257958
400: 2.8865945839724438e-06,-2.017181303564803
401: 0.7639320347647365,-1.7212300777221412
402: 4.4887764172038045,7.702961832840413
403: -9.161955296290891,-10.593567064441348
404: 24.17849149104874,0.10471747551975552
405: -16.43853222788647,1.5857471140467498
406: 1.9621960435780055,0.7627457841109276
407: -3.4320984431121144,5.193846753519293
408: -4.243144145696706,2.2614120397691035
409: -0.7841344097414058,1.6220091205750042
410: 2.1352

579: 0.5394022876186269,2.086891744952405
580: 1.1666357627233264,2.849306867152919
581: 4.687656774470647,4.235596973424611
582: -0.7536579461623436,3.0660224480960467
583: 2.586436071969636,-0.6944175338729555
584: 0.17673571355477513,2.618033974844
585: 0.19329409306101936,2.506908610588358
586: -1.4132841530717655,5.909808637506542
587: 5.334368531284322,1.0181278082495162
588: 3.490553086028772,-0.25269305361426575
589: 1.9999999844527365,-0.013455449338086794
590: -3.209149982264047,-7.518410252516059
591: 0.009266922374284225,2.0104641009465354
592: 0.5374249646545293,-1.639320193576634
593: -3.306591078710014,2.766972145150784
594: 6.763709282767481,-0.42665123138383554
595: 4.40988899527259,3.0362650333475747
596: -2.575915050294771,-2.82413065433079
597: 1.7965148415882506,3.483318597218886
598: 1.3668093047417518,4.888801495428552
599: -0.07153430085339352,-1.0229456837828361
600: -0.5481507720130955,5.27059279698622
601: -2.7077785805052983,0.29813397005432707
602: 3.521810

771: 0.41995365872950613,3.6081672484287806
772: 4.632610086338772,-6.4828606879571495
773: -0.4518830509837902,3.6588518815102735
774: 2.430267899829395,-3.0642597413648485
775: 0.2023958272165187,1.046495189858121
776: -10.99877622116427,2.4159197827184355
777: 1.0007678218117897,2.090169926800315
778: -1.9619897017287689,-0.22934960746327954
779: 5.488372406710945,3.7095593976571735
780: 1.183486364376631,2.4455007669623883
781: 1.6023924991131262,4.003654802782309
782: 1.1564837557156633,-3.548344134062515
783: 1.2343740297231536,-1.470717617461489
784: -4.093748518382219,0.38388538807826866
785: 9.838894288708165,7.123500101529487
786: 0.38415001258407677,-0.09570547045884881
787: -0.3104287124739018,1.731209783152297
788: 0.3572633710294873,0.6792801636176685
789: 2.0453101631145856,1.7324351650289416
790: -0.09536554435421918,2.1934954947380954
791: -0.618033974844,-1.1763506181301182
792: 20.17052583626126,8.648543284123374
793: 3.4350664376236866,6.5862229845201
794: 11.429097

...

9739: 6.255965477378412,1.0965886763757302
9740: 1.0,-1.1990772346434193
9741: 3.4045829362145517,-2.941796084976838
9742: 0.7651508876867645,-1.4622641620301127
9743: 4.802090528607227,1.3252396495383496
9744: -3.2536366930050398,4.383525506515142
9745: 3.215937622085071,3.3012435563203675
9746: 2.6233610543565553,1.7733264724586775
9747: -0.473161075857918,-0.8672952514237109
9748: 3.2157243519120726,1.2055355919835968
9749: 1.448915632735243,4.9202985381124025
9750: -0.8319781540101949,2.7650996283057157
9751: 2.1714168261957045,-2.107068695492953
9752: 4.059367013248648,0.7295481616306154
9753: 7.716322265094393,4.83600257932833
9754: -8.157459881119767,0.08165594716317603
9755: 1.6595232409068794,1.6490810021091966
9756: -0.08341186628666508,0.17558160919567162
9757: 2.501329225142061,6.022385792919088
9758: -5.792385625463361,3.6180339999999998
9759: 9.979461998116241,3.446536147888788
9760: 1.6592402218474257e-09,1.618033974844
9761: 5.573107877019535,-1.4373468008432955
9762: -

9926: 2.2357960170262037,-2.2853520164704904
9927: 3.152793281160477,6.282166211970278
9928: 0.7639320096087368,1.0198840321296936
9929: 4.009575288586312,4.853883267715261
9930: 4.8606046701303605,5.4354132275791285
9931: 6.908534031573447,2.4373484899810083
9932: -0.0038387568424952355,1.1754045301641716
9933: 2.0507328618915026,0.7082039208617893
9934: 0.004050337109530738,4.0425144051083794
9935: -3.283657744245671,1.3867704319563716
9936: -5.29014853866321,4.39521703756868
9937: 0.5863420069369802,4.959701373317684
9938: 0.7765891036970769,-1.1038994518877485
9939: -0.6872025082872464,-0.4457535168469719
9940: 6.699636647238888,8.614344964893649
9941: -7.525537119548785,4.228656431455198
9942: -3.0852334368587635,1.8166093665660314
9943: -1.9048152718486813,-3.4364540078811787
9944: 1.1378743995032092,2.393332614093642
9945: 0.21733868725935016,1.6146409083432136
9946: 2.5964677166446073,1.2777227699090818
9947: -2.8506455630986984,-0.20147130600033747
9948: -1.097603035998686,-0.

10110: 2.238612518109041,-1.0299440460578242
10111: -1.1470868837892698,7.634669993370065
10112: -3.569179579397329,1.674249239438826
10113: -0.6126777651864999,0.6130090019078985
10114: 2.2913737138741324,0.7079465223310605
10115: 2.618033974844,2.971860509276312
10116: 0.5078895838649296,2.4090599747693675
10117: -0.6102635700895847,-14.936821273132544
10118: 2.4022870614433236,0.11456276569426027
10119: 12.402933853416293,-1.2171384423927445
10120: 6.000184705189895,1.2691544009226814
10121: 4.840609692746693,0.8514558325258921
10122: 12.129072542403994,3.793358820493046
10123: 5.276739359807371,-0.9757669892984584
10124: 3.7906076509919213,-0.4342044933469176
10125: 6.60282044994891,0.5902770446650957
10126: 0.6210822880125432,1.5740153873113876
10127: 4.145045792031839,9.082612299026653
10128: 3.128017531806438,-4.387822554837683
10129: 2.6565579555963756,1.0477093743143895
10130: -1.0007177776670808,1.5520819848986807
10131: 0.01301714922426711,2.067216676565575
10132: 2.32110443

10292: 0.30970645844737255,-1.9712181945381084
10293: 2.9960636374964107,-2.5113187786377855
10294: -0.9471736989449576,-2.258283040615875


KeyboardInterrupt: 

In [ ]:
shift_processed_path = "E:\\Data\\anat0mixer_temp\\shifter_processed"
for n in range(len(x_train_shifted)):
    np.save(shift_processed_path+f"\\x_train_shifted_{n}", 
            x_train_shifted[n])

In [ ]:
for n in range(len(x_test_shifted)):    
    np.save(shift_processed_path+f"\\x_test_shifted_{n}", 
            x_test_shifted[n])

In [ ]:
from pathlib import Path
x_train_fns = \
    Path('E:\\Data\\anat0mixer_temp\\shifter_processed') \
        .glob('x_train_shifted_*.npy')
x_train_shifted = []
for fn in x_train_fns:
    x_train_shifted.append(np.load(fn))

x_test_fns = \
    Path('E:\\Data\\anat0mixer_temp\\shifter_processed') \
        .glob('x_test_shifted_*.npy')
x_test_shifted = []
for fn in x_test_fns:
    x_test_shifted.append(np.load(fn))

In [ ]:
x_train_shifted = np.array(x_train_shifted)
x_test_shifted = np.array(x_test_shifted)
x_train_shifted.shape, x_test_shifted.shape

In [ ]:
autoencoder.fit(x_train_shifted, x_train_shifted, 
                epochs=256, batch_size=128, 
                shuffle=True, validation_data=(x_test_shifted, x_test_shifted))

In [ ]:
import tensorflow as tf
autoencoder.save_weights('E:\\Data\\anat0mixer_temp\\autoencoder_v1')
encoder.save_weights('E:\\Data\\anat0mixer_temp\\encoder_v1')
decoder.save_weights('E:\\Data\\anat0mixer_temp\\decoder_v1')

In [ ]:
encoded_shift = encoder.predict(x_test_shifted[80:90])
decoded_imgs_shift = \
    [decode_latent(decoder, tuple(latent_vector)) 
         for latent_vector in encoded_shift[2]]
show_original_decoded(x_test_shifted[80:90], decoded_imgs_shift, sz)

## Saving the Model
If the model output looks good, then save it for subsequent use

In [ ]:
encoder.save('data\zebrastack_v0_covidnet_encoder_model')
decoder.save('data\zebrastack_v0_covidnet_decoder_model')

## Anat0Mixer Interaction

In [ ]:
def update_image_for_sliders(**kwargs):
    from scipy.ndimage import zoom
    # print(tuple(kwargs.values()))
    decoded_pixel_array = decode_latent(decoder, tuple(kwargs.values()))
    interp_array = decoded_pixel_array.reshape(sz,sz)
    # interp_array = zoom(decoded_pixel_array.reshape(sz,sz), 4.0, order=0)
    # interp_array = interp_array.reshape(sz*4, sz*4)
    # ax.imshow(interp_array, cmap='gray')
    # a=im.get_array()    
    im.set_array(interp_array)
    fig.canvas.draw()
    return kwargs['1']

In [ ]:
%matplotlib notebook
from ipywidgets import FloatSlider, interactive_output, Layout, HBox, VBox

kwargs = {}

for k in range(latent_dim):
    kwargs[str(k)] = \
        FloatSlider(value = 0,
                    min = -2.0, max = 2.0, 
                    step = 0.01,
                    orientation='vertical',
                    layout=Layout(padding='0%'))

decoded_pixel_array = decode_latent(decoder, tuple([slider.value for slider in kwargs.values()]))
interp_array = decoded_pixel_array.reshape(sz,sz)


In [ ]:
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot()
im = ax.imshow(interp_array, cmap='gray')

slider_widgets = interactive_output(update_image_for_sliders, kwargs)
slider_widgets.layout.height = '350px'

import gc
gc.disable()
display(HBox(list(kwargs.values()), layout = Layout(padding='0%')))
display(slider_widgets)

In [ ]:
gc.enable()